In [ ]:
!pip install tokenizers==0.9.4
!pip install simpletransformers

In [1]:
from transformers import DistilBertTokenizer, AdamW, DistilBertModel, DistilBertConfig
import pandas as pd
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
import torch.nn as nn

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


## Read data from csv files

In [3]:
def read_train_and_test(train_path, test_path): #path: path of the whole dataset
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)
    return train, test

train_path = "../input/re-balance-train/re_balance_train2.csv"
test_path = "../input/raw-train-test/test.csv"
train_csv, test_csv = read_train_and_test(train_path, test_path)

In [4]:
train_labels = list(train_csv.label.values)
print(train_labels.count(0))
print(train_labels.count(1))
print(train_labels.count(2))

1136
1136
1136


In [5]:
print(train_csv.tweet.values[:10])

['@LandonVanBus Charlie Day?'
 "omg RT @SaddyBey: Fat bitch. What's her @? http://t.co/ptNszx6nid"
 'RT @LizardLickTowin: - @CMchatLIVE #CMchat why do U think its so hard for"hick hop"2get major air time on most stations #cmchat when it\'s w&#8230;'
 'RT @CHlLDHOODRUINER: "Great.... DaQuan got her pregnant and now I\'m gonna have a nigger grandbaby" http://t.co/9aAyrrKNbt'
 "Trust none bitch I don't wife a bitch? Bitch I won't"
 'RT @ItsFoodPorn: Oreo cheesecake bites! http://t.co/iHgu1ZuyZt'
 'DIVERSITY: Chcgo Blck Pstr Flooded w/ Dth Threats Ovr Endorsement Of GOP Candidate: &#8220;U Sellout Uncle Tom Ass N*gger&#8221; http://t.co/vcBWzkyeDv'
 '"All these bitches want a baby, I don\'t want no children."'
 'I know Outback when I see Outback, hoe.'
 'RT @SteveStfler: ur a faggot if you change your name to your bday just so you can get bday tweets. this aint FB nigga, fuck yo birthday']


## Run tokenizer

In [6]:
def run_tokenizer(train_csv, test_csv, merge_label=False, add_token=False):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased') 
    if add_token:
      tokenizer.add_tokens(add_token)

    def get_max_len(tokenizer, train_csv):
        tweets = train_csv.tweet.values
        max_length = 0
        for t in tweets:
          ids = tokenizer.encode(t)
          max_length = max(len(ids),max_length)
        return max_length

    max_length = get_max_len(tokenizer, train_csv)
    train_tweets, train_labels = train_csv.tweet.values, train_csv.label.values
    test_tweets, test_labels = test_csv.tweet.values, test_csv.label.values
    if merge_label == True:
       train_labels = [l if l ==0 else 1 for l in train_labels]
       test_labels = [l if l ==0 else 1 for l in test_labels]

    def tokenize_for_tweet(tokenizer, tweets, labels):
        input_ids = []
        attention_masks = []

        for t in tweets:
            input_dict = tokenizer.encode_plus(t, add_special_tokens=True, max_length=max_length, truncation=True, padding='max_length',return_tensors='pt')
            input_ids.append(input_dict['input_ids'])
            attention_masks.append(input_dict['attention_mask'])
        input_ids = torch.cat(input_ids,dim=0)
        attention_masks = torch.cat(attention_masks,dim=0)
        labels=torch.tensor(labels)
        dataset = TensorDataset(input_ids, attention_masks, labels)
        return dataset
        
    train_dataset = tokenize_for_tweet(tokenizer, train_tweets, train_labels)
    test_dataset = tokenize_for_tweet(tokenizer, test_tweets, test_labels)
    num_label = 3 if merge_label == False else 2
    return train_dataset, test_dataset, num_label, tokenizer

#tokens = ['<SWEAR-0>', '<SWEAR-1>', '<SWEAR-2>', '<SWEAR-3>', '<SWEAR-4>']
train_dataset, test_dataset, num_label, tokenizer = run_tokenizer(train_csv, test_csv, merge_label = False)#,add_token = tokens)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

## Make data loader

In [7]:
batch_size = 20

train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)

test_dataloader = DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = batch_size)

## Define classifier and adversary

In [8]:
configuration = DistilBertConfig()
#configuration.output_hidden_states = True
#print(configuration)

In [9]:
class Classifier(nn.Module):
  def __init__(self, num_label):
    super().__init__()
    self.bert = DistilBertModel.from_pretrained('distilbert-base-cased')
    #self.bert.resize_token_embeddings(len(tokenizer))
    self.linear = nn.Linear(configuration.hidden_size, num_label)

  def forward(self, input_ids, attention_mask): # input_id [batch_size, sentence_length]
    last_hidden_state = self.bert(input_ids, attention_mask)[0] # last_hidden_state [batch_size, sentence_length, hidden_size]
    last_hidden_state_mean = torch.mean(last_hidden_state, dim=1) # last_hidden_state [batch_size, hidden_size]
    output = self.linear(last_hidden_state_mean) # output [batch_size, num_label]
    return last_hidden_state_mean, output


In [ ]:
class Adversary_lstm(nn.Module):
  def __init__(self, num_protected_label, hidden_size):
    super().__init__()
    self.lstm = nn.LSTM(configuration.hidden_size, hidden_size,bidirectional=True, batch_first=True)
    #self.relu = nn.ReLU()
    self.linear = nn.Linear(hidden_size*2, num_protected_label)

  def forward(self, clf_input):
    #clf_input = torch.unsqueeze(clf_input, dim=1)
    lstm, _ = self.lstm(clf_input)
    #lstm = torch.squeeze(lstm, dim=1)
    lstm = lstm[:,-1,:]
    #output1 = self.relu(lstm)
    output = self.linear(lstm)
    return output

In [10]:
class Adversary(nn.Module):
  def __init__(self, num_protected_label, hidden_size):
    super().__init__()
    self.linear1 = nn.Linear(configuration.hidden_size, hidden_size)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(hidden_size, num_protected_label)

  def forward(self, clf_last_state):
    output1 = self.relu(self.linear1(clf_last_state))
    output = self.linear2(output1)
    return output

## Import some tools for evaluation after each training epoch

In [11]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
import numpy as np
import random

In [12]:
def plot_confusion_matrix(y_true, y_pred, classes, normalize=False, title=None, cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [13]:
def print_matrix(true_labels, predict_labels, default_classes=['hate','offensive','neither']):
  cm = confusion_matrix(true_labels, predict_labels)
  plot_confusion_matrix(true_labels, predict_labels, classes=default_classes ,title='Confusion matrix with rebalance')

  #Plot normalized confusion matrix
  plot_confusion_matrix(true_labels, predict_labels, classes=default_classes, normalize=True,title='Normalized confusion matrix with rebalance')

  plt.show()

  print(classification_report(true_labels, predict_labels, target_names=default_classes))

## Training and evaluation function

In [14]:
def joint_training(clf, adv, epochs, clf_optimizer, adv_optimizer, train_dataloader, test_dataloader, alpha, model_name):
    best_acc = 0
    for e in range(epochs):
        print('training {} epoch...'.format(e+1))
        start_time = time.time()

        train_loss, total_clf_loss, total_adv_loss = 0, 0, 0

        clf.train(True)
        adv.train(True)
        for input, mask, label in train_dataloader:
            input = input.to(DEVICE)
            mask = mask.to(DEVICE)
            label=label.to(DEVICE)

            protected_label = torch.tensor([1 if l == 1 else 0 for l in label], dtype=torch.long).to(DEVICE) # 0 no-offensive 1 offensive
            
            clf.zero_grad()
            adv.zero_grad()

            last_hidden_state, clf_output = clf(input_ids=input, attention_mask=mask)

            adv_output = adv(last_hidden_state)

            clf_loss = loss_function(clf_output, label)
            adv_loss = loss_function(adv_output, protected_label)

            total_loss = clf_loss + alpha*adv_loss

            train_loss += total_loss.item()
            total_clf_loss += clf_loss.item()
            total_adv_loss += adv_loss.item()

            total_loss.backward(retain_graph=True)
            clf_optimizer.step()

            adv_loss.backward()
            adv_optimizer.step()
            

        avg_train_loss = train_loss / len(train_dataloader)
        sec = time.time()-start_time
        print('{} seconds used......'.format(sec))
        print("{} training finished! average train loss: {}".format(e+1,avg_train_loss))
        print('total clf loss: {} total adv loss: {}'.format(total_clf_loss, total_adv_loss))
        print('evaluating...')
        best_acc = evaluate(clf, best_acc, test_dataloader, model_name, str(e+1))
        #print('evaluating adversary...')
        #evaluate_adv(adv, clf, test_dataloader)
        #torch.save(adv,"adv"+str(e+1))
        

In [15]:
def evaluate(clf, best_acc, test_dataloader, model_name, epoch):
    num_total, num_correct = 0, 0
    clf.train(False)
    with torch.no_grad():
      eval_loss = 0
      true_labels, predict_labels = [], []
      for input, mask, label in test_dataloader:
          clf.zero_grad()
          
          input = input.to(DEVICE)
          mask = mask.to(DEVICE)
          label = label.to(DEVICE)

          last_hidden_state, output = clf(input_ids=input, attention_mask=mask)

          loss = loss_function(output, label)

          predict_label = torch.argmax(output, dim=1)

          true_labels += label.tolist()
          predict_labels += predict_label.tolist()

          num_correct += (predict_label == label).sum().item()
          num_total += len(label)

          eval_loss += loss.item()

      avg_eval_loss = eval_loss / len(test_dataloader)

      acc = num_correct/num_total
      if acc > best_acc:
        best_acc = acc
      torch.save(clf, epoch+model_name)
      print_matrix(true_labels, predict_labels)

    print('average eval_loss: {}, accuracy: {}'.format(avg_eval_loss,acc))
    return best_acc


In [ ]:
def evaluate_adv(adv, clf, test_dataloader):
    clf.train(False)
    adv.train(False)
    with torch.no_grad():
        true_labels, predict_labels = [], []
        for input, mask, label in test_dataloader:
            input = input.to(DEVICE)
            mask = mask.to(DEVICE)
            label=label.to(DEVICE)

            protected_label = torch.tensor([1 if l == 1 else 0 for l in label], dtype=torch.long).to(DEVICE) # 0 no-offensive 1 offensive
            
            clf.zero_grad()
            adv.zero_grad()

            last_hidden_state, clf_output = clf(input_ids=input, attention_mask=mask)

            adv_output = adv(last_hidden_state)

            predict_label = torch.argmax(adv_output, dim=1)
            true_labels += protected_label.tolist()
            predict_labels += predict_label.tolist()
        print_matrix(true_labels, predict_labels, default_classes=['no offensive','offensive']) 
             

## Define the classifier and adversary

In [16]:
def train_clf(clf, epochs, clf_optimizer, train_dataloader, test_dataloader, model_name):
    best_acc = 0
    for e in range(epochs):
        print('training {} epoch...'.format(e+1))
        start_time = time.time()

        train_loss = 0

        clf.train(True)
        for input, mask, label in train_dataloader:
            input = input.to(DEVICE)
            mask = mask.to(DEVICE)
            label=label.to(DEVICE)
            
            clf.zero_grad()

            last_hidden_state, clf_output = clf(input_ids=input, attention_mask=mask)
                       
            clf_loss = loss_function(clf_output, label)
            
            train_loss += clf_loss.item()

            clf_loss.backward()
            clf_optimizer.step()

        avg_train_loss = train_loss / len(train_dataloader)
        sec = time.time()-start_time
        print('{} seconds used......'.format(sec))
        print("{} training finished! average train loss: {}".format(e+1,avg_train_loss))
        print('evaluating...')
        best_acc = evaluate(clf, best_acc, test_dataloader, model_name, str(e+1))

In [23]:
classifier = Classifier(num_label).to(DEVICE)

#classifier = torch.load('./1distilbert_clf')

adversary = Adversary(2, 200).to(DEVICE)

loss_function = nn.CrossEntropyLoss()

clf_optimizer = AdamW(classifier.parameters(),lr = 2e-5, eps = 1e-8)

adv_optimizer = torch.optim.AdamW(adversary.parameters(), lr=0.001)

In [ ]:
train_clf(classifier, 10, clf_optimizer, train_dataloader, test_dataloader, 'distilbert_clf')

training 1 epoch...


## Joint training
In this step, only the total loss which is defined as L = L<sub>clf</sub> − α∗L<sub>adv</sub> is used for backpropagation. Hence, α is also a import hyperparameter. 

In [ ]:
epochs = 10
alpha = 2
model_name = 'adv_hate_as_noffensive'

joint_training(classifier, adversary, epochs, clf_optimizer, adv_optimizer, train_dataloader, test_dataloader, alpha, model_name)

## Rebalance the number of instances in classes
In the prevous experiment, rebalancing is an effective method to increase the true positive rate for hate speech class. (However, the scores of other classes will decrease)

In [ ]:
def rebalance_dataset(dataset):
    data_0 = [inst for inst in dataset if inst[2]==0]
    data_1 = [inst for inst in dataset if inst[2]==1]
    data_2 = [inst for inst in dataset if inst[2]==2]
    random.shuffle(data_0)
    random.shuffle(data_1)
    random.shuffle(data_2)
    num_data0 = len(data_0)
    balanced_dataset = data_0+data_1[:num_data0]+data_2[:num_data0]
    return balanced_dataset

In [ ]:
re_train_dataset = rebalance_dataset(train_dataset)
re_test_dataset = rebalance_dataset(test_dataset)

In [ ]:
batch_size = 20

In [ ]:
re_train_dataloader = DataLoader(re_train_dataset, sampler = RandomSampler(re_train_dataset), batch_size = batch_size)
re_test_dataloader = DataLoader(re_test_dataset, sampler = RandomSampler(re_test_dataset), batch_size = batch_size)

In [ ]:
epochs = 10
alpha = 2
model_name = 'adv_model_hate_as_noffensive'
joint_training(classifier, adversary, epochs, clf_optimizer, adv_optimizer, re_train_dataloader, test_dataloader, alpha, model_name)